# 75 persons
## electrodes : 8, 16, 32, 64

In [1]:
from glob import glob
import os
import mne
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from xgboost import XGBClassifier

2023-12-01 07:24:05.867321: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-01 07:24:05.898426: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-01 07:24:05.898455: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-01 07:24:05.899512: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-01 07:24:05.904863: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-01 07:24:05.905723: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
no_of_patients=75

In [3]:
train=sorted(glob('files2/*.edf'))
train=train[:no_of_patients]
train

['files2/S001R04.edf',
 'files2/S002R04.edf',
 'files2/S003R04.edf',
 'files2/S004R04.edf',
 'files2/S005R04.edf',
 'files2/S006R04.edf',
 'files2/S007R04.edf',
 'files2/S008R04.edf',
 'files2/S009R04.edf',
 'files2/S010R04.edf',
 'files2/S011R04.edf',
 'files2/S012R04.edf',
 'files2/S013R04.edf',
 'files2/S014R04.edf',
 'files2/S015R04.edf',
 'files2/S016R04.edf',
 'files2/S017R04.edf',
 'files2/S018R04.edf',
 'files2/S019R04.edf',
 'files2/S020R04.edf',
 'files2/S021R04.edf',
 'files2/S022R04.edf',
 'files2/S023R04.edf',
 'files2/S024R04.edf',
 'files2/S025R04.edf',
 'files2/S026R04.edf',
 'files2/S027R04.edf',
 'files2/S028R04.edf',
 'files2/S029R04.edf',
 'files2/S030R04.edf',
 'files2/S031R04.edf',
 'files2/S032R04.edf',
 'files2/S033R04.edf',
 'files2/S034R04.edf',
 'files2/S035R04.edf',
 'files2/S036R04.edf',
 'files2/S037R04.edf',
 'files2/S038R04.edf',
 'files2/S039R04.edf',
 'files2/S040R04.edf',
 'files2/S041R04.edf',
 'files2/S042R04.edf',
 'files2/S043R04.edf',
 'files2/S0

In [4]:
train_split=0.8

In [5]:
def read_data(i,train_split,valid_split):
    raw = mne.io.read_raw_edf(i, preload=True)
    eeg_data = raw.get_data()
    eeg_channels = [f'Channel_{i}' for i in range(eeg_data.shape[0])]
    eeg_df = pd.DataFrame(data=eeg_data.T, columns=eeg_channels)
    
    eeg_df = eeg_df.iloc[:15000]
    eeg_df.sample(frac=1)
    
    idx1= int(train_split*(len(eeg_df)))
    idx2= int(train_split*(len(eeg_df)))+1
    eeg_df1=eeg_df.iloc[:idx1]
    eeg_df2=eeg_df.iloc[idx2:]
    idx3=int(valid_split*(len(eeg_df2)))
    idx4=int(valid_split*(len(eeg_df2)))+1
    eeg_df3=eeg_df2.iloc[:idx3]
    eeg_df4=eeg_df2.iloc[idx4:]
    return eeg_df1,eeg_df3,eeg_df4,len(eeg_df1),len(eeg_df3),len(eeg_df4)

    

In [6]:
%%capture
xtemp1=[]
xtemp2=[]
xtemp3=[]
ytemp1=[]
ytemp2=[]
ytemp3=[]
for i in range(no_of_patients):
    xtr,xte,xval,ytr,yte,yval=read_data(train[i],0.8,0.5) # xtr=xtrain, xte=xtest, ytr=ytrain, yte=ytest.
    xtemp1.append(xtr)
    xtemp2.append(xte)
    xtemp3.append(xval)
    ytemp1.append(ytr)
    ytemp2.append(yte)
    ytemp3.append(yval)

In [7]:
xtrain = pd.concat([xtemp1[i] for i in range(0, len(xtemp1))], ignore_index=True)
xtest = pd.concat([xtemp2[i] for i in range(0, len(xtemp2))], ignore_index=True)
xvalid=pd.concat([xtemp3[i] for i in range(0,len(xtemp3))],ignore_index=True)

In [8]:
ytrain=[]
for i in range(len(ytemp1)):
    for j in range(ytemp1[i-1]):
        ytrain.append(i)
ytest=[]
for i in range(len(ytemp2)):
    for j in range(ytemp2[i-1]):
        ytest.append(i)        
yvalid=[]
for i in range(len(ytemp3)):
    for j in range(ytemp3[i-1]):
        yvalid.append(i)     

In [9]:
len(xtrain),len(xtest),len(ytrain),len(ytest)

(900000, 112425, 900000, 112425)

In [10]:
print(xtest.iloc[:,-1].values)

[-1.0e-05  1.5e-05  6.0e-06 ... -1.6e-05 -1.3e-05 -6.0e-06]


In [11]:
display(xtrain)

,Channel_0,Channel_1,Channel_2,Channel_3,Channel_4,Channel_5,Channel_6,Channel_7,Channel_8,Channel_9,...,Channel_54,Channel_55,Channel_56,Channel_57,Channel_58,Channel_59,Channel_60,Channel_61,Channel_62,Channel_63
0,-0.000005,0.000002,0.000037,0.000039,0.000030,0.000026,-0.000016,-0.000014,0.000004,0.000018,...,-0.000021,-0.000008,-0.000035,-0.000045,-0.000066,-0.000039,-0.000033,-0.000048,-0.000039,-0.000039
1,-0.000012,-0.000024,0.000001,-0.000002,-0.000015,-0.000022,-0.000055,-0.000036,-0.000027,-0.000025,...,-0.000050,-0.000040,-0.000068,-0.000065,-0.000084,-0.000052,-0.000021,-0.000042,-0.000031,-0.000034
2,-0.000077,-0.000078,-0.000059,-0.000065,-0.000063,-0.000055,-0.000067,-0.000088,-0.000071,-0.000065,...,-0.000017,-0.000022,-0.000050,-0.000035,-0.000048,-0.000018,-0.000020,-0.000042,-0.000029,-0.000027
3,-0.000066,-0.000067,-0.000050,-0.000065,-0.000060,-0.000055,-0.000068,-0.000062,-0.000053,-0.000054,...,-0.000039,-0.000060,-0.000078,-0.000064,-0.000068,-0.000041,-0.000044,-0.000062,-0.000034,-0.000043
4,-0.000045,-0.000055,-0.000033,-0.000053,-0.000054,-0.000063,-0.000083,-0.000052,-0.000050,-0.000053,...,-0.000044,-0.000055,-0.000070,-0.000054,-0.000063,-0.000037,-0.000060,-0.000070,-0.000034,-0.000045
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899995,0.000055,0.000013,-0.000036,-0.000092,-0.000044,-0.000073,0.000019,0.000028,0.000032,-0.000043,...,0.000005,0.000019,-0.000023,-0.000062,-0.000021,-0.000005,0.000022,0.000022,-0.000018,0.000014
899996,-0.000050,-0.000065,-0.000089,-0.000119,-0.000064,-0.000116,-0.000026,-0.000044,-0.000070,-0.000080,...,-0.000007,-0.000014,-0.000038,-0.000075,-0.000048,-0.000031,-0.000008,0.000012,-0.000035,-0.000030
899997,0.000001,-0.000021,-0.000052,-0.000108,-0.000053,-0.000102,-0.000020,-0.000034,-0.000003,-0.000039,...,-0.000044,0.000030,0.000022,-0.000021,-0.000030,-0.000039,0.000026,0.000058,0.000017,0.000050
899998,0.000011,0.000051,0.000003,-0.000079,-0.000023,-0.000072,0.000031,-0.000010,0.000079,0.000008,...,-0.000064,0.000006,0.000007,-0.000009,-0.000026,-0.000063,0.000028,0.000044,-0.000009,0.000039


In [12]:
def scale_dataset(dataframe):
    x=dataframe.iloc[:,:-1].values
    y=dataframe.iloc[:,-1].values
    scaler =StandardScaler()
    x=scaler.fit_transform(x)
    return x,y

## 0-8

In [13]:
xtrain8=xtrain.iloc[:,:8]
xvalid8=xvalid.iloc[:,:8]
xtest8=xtest.iloc[:,:8]

In [14]:
xtrain8['id']=ytrain
xtest8['id']=ytest
xvalid8['id']=yvalid
display(xtrain)

/tmp/ipykernel_78467/394288547.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtrain8['id']=ytrain
/tmp/ipykernel_78467/394288547.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtest8['id']=ytest
/tmp/ipykernel_78467/394288547.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-

,Channel_0,Channel_1,Channel_2,Channel_3,Channel_4,Channel_5,Channel_6,Channel_7,Channel_8,Channel_9,...,Channel_54,Channel_55,Channel_56,Channel_57,Channel_58,Channel_59,Channel_60,Channel_61,Channel_62,Channel_63
0,-0.000005,0.000002,0.000037,0.000039,0.000030,0.000026,-0.000016,-0.000014,0.000004,0.000018,...,-0.000021,-0.000008,-0.000035,-0.000045,-0.000066,-0.000039,-0.000033,-0.000048,-0.000039,-0.000039
1,-0.000012,-0.000024,0.000001,-0.000002,-0.000015,-0.000022,-0.000055,-0.000036,-0.000027,-0.000025,...,-0.000050,-0.000040,-0.000068,-0.000065,-0.000084,-0.000052,-0.000021,-0.000042,-0.000031,-0.000034
2,-0.000077,-0.000078,-0.000059,-0.000065,-0.000063,-0.000055,-0.000067,-0.000088,-0.000071,-0.000065,...,-0.000017,-0.000022,-0.000050,-0.000035,-0.000048,-0.000018,-0.000020,-0.000042,-0.000029,-0.000027
3,-0.000066,-0.000067,-0.000050,-0.000065,-0.000060,-0.000055,-0.000068,-0.000062,-0.000053,-0.000054,...,-0.000039,-0.000060,-0.000078,-0.000064,-0.000068,-0.000041,-0.000044,-0.000062,-0.000034,-0.000043
4,-0.000045,-0.000055,-0.000033,-0.000053,-0.000054,-0.000063,-0.000083,-0.000052,-0.000050,-0.000053,...,-0.000044,-0.000055,-0.000070,-0.000054,-0.000063,-0.000037,-0.000060,-0.000070,-0.000034,-0.000045
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899995,0.000055,0.000013,-0.000036,-0.000092,-0.000044,-0.000073,0.000019,0.000028,0.000032,-0.000043,...,0.000005,0.000019,-0.000023,-0.000062,-0.000021,-0.000005,0.000022,0.000022,-0.000018,0.000014
899996,-0.000050,-0.000065,-0.000089,-0.000119,-0.000064,-0.000116,-0.000026,-0.000044,-0.000070,-0.000080,...,-0.000007,-0.000014,-0.000038,-0.000075,-0.000048,-0.000031,-0.000008,0.000012,-0.000035,-0.000030
899997,0.000001,-0.000021,-0.000052,-0.000108,-0.000053,-0.000102,-0.000020,-0.000034,-0.000003,-0.000039,...,-0.000044,0.000030,0.000022,-0.000021,-0.000030,-0.000039,0.000026,0.000058,0.000017,0.000050
899998,0.000011,0.000051,0.000003,-0.000079,-0.000023,-0.000072,0.000031,-0.000010,0.000079,0.000008,...,-0.000064,0.000006,0.000007,-0.000009,-0.000026,-0.000063,0.000028,0.000044,-0.000009,0.000039


In [15]:
x8,y8=scale_dataset(xtrain8)
xt8,yt8=scale_dataset(xtest8)
xv8,yv8=scale_dataset(xvalid8)

In [16]:
model=XGBClassifier(n_estimators=500)
model.fit(x8,y8,early_stopping_rounds=10, eval_set=[(xv8, yv8)])
y_pred=model.predict(xt8)
print(classification_report(yt8,y_pred))

/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-mlogloss:4.03047
[1]	validation_0-mlogloss:3.91937
[2]	validation_0-mlogloss:3.82976
[3]	validation_0-mlogloss:3.76401
[4]	validation_0-mlogloss:3.70940
[5]	validation_0-mlogloss:3.66650
[6]	validation_0-mlogloss:3.62728
[7]	validation_0-mlogloss:3.59370
[8]	validation_0-mlogloss:3.56536
[9]	validation_0-mlogloss:3.54245
[10]	validation_0-mlogloss:3.52024
[11]	validation_0-mlogloss:3.50060
[12]	validation_0-mlogloss:3.48300
[13]	validation_0-mlogloss:3.46688
[14]	validation_0-mlogloss:3.45414
[15]	validation_0-mlogloss:3.43916
[16]	validation_0-mlogloss:3.42744
[17]	validation_0-mlogloss:3.41602
[18]	validation_0-mlogloss:3.40528
[19]	validation_0-mlogloss:3.39415
[20]	validation_0-mlogloss:3.38442
[21]	validation_0-mlogloss:3.37375
[22]	validation_0-mlogloss:3.36319
[23]	validation_0-mlogloss:3.35296
[24]	validation_0-mlogloss:3.34590
[25]	validation_0-mlogloss:3.33921
[26]	validation_0-mlogloss:3.32994
[27]	validation_0-mlogloss:3.32020
[28]	validation_0-mlogloss:3.3

In [16]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model = Sequential(
    [
        Dense(8, activation = 'relu',   name = "L1"),
        Dense(256, activation = 'relu', name = "L2"),
        Dense(128, activation = 'relu', name = "L3"),
        Dense(64, activation = 'relu',   name = "L4"),
        Dense(32, activation = 'relu', name = "L5"),
        Dense(75, activation = 'linear', name = "L6"),
    ]
)

In [17]:
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model.fit(
    x8,y8,epochs=10,validation_data=(xv8,yv8)
)

Epoch 1/10
28125/28125 [==============================] - 79s 3ms/step - loss: 2.7215 - val_loss: 2.8924
Epoch 2/10
28125/28125 [==============================] - 84s 3ms/step - loss: 2.2078 - val_loss: 2.9173
Epoch 3/10
28125/28125 [==============================] - 85s 3ms/step - loss: 2.0909 - val_loss: 2.9269
Epoch 4/10
28125/28125 [==============================] - 75s 3ms/step - loss: 2.0340 - val_loss: 3.0103
Epoch 5/10
28125/28125 [==============================] - 60s 2ms/step - loss: 1.9979 - val_loss: 3.0350
Epoch 6/10
28125/28125 [==============================] - 62s 2ms/step - loss: 1.9729 - val_loss: 2.9166
Epoch 7/10
28125/28125 [==============================] - 63s 2ms/step - loss: 1.9533 - val_loss: 3.0732
Epoch 8/10
28125/28125 [==============================] - 61s 2ms/step - loss: 1.9357 - val_loss: 3.0248
Epoch 9/10
28125/28125 [==============================] - 63s 2ms/step - loss: 1.9230 - val_loss: 3.1574
Epoch 10/10
28125/28125 [==============================

In [18]:
y_pred_classes = np.argmax(tf.nn.softmax(model.predict(xt8)).numpy(),axis=1)
print(classification_report(yt8,y_pred_classes))

3514/3514 [==============================] - 5s 1ms/step


2023-10-04 13:47:17.275148: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 33727500 exceeds 10% of free system memory.
2023-10-04 13:47:21.076335: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 33727500 exceeds 10% of free system memory.
2023-10-04 13:47:21.083214: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 33727500 exceeds 10% of free system memory.


              precision    recall  f1-score   support

           0       0.14      0.23      0.17      1499
           1       0.21      0.18      0.20      1499
           2       0.21      0.23      0.22      1499
           3       0.42      0.32      0.36      1499
           4       0.67      0.67      0.67      1499
           5       0.23      0.25      0.24      1499
           6       0.19      0.36      0.25      1499
           7       0.36      0.28      0.31      1499
           8       0.31      0.37      0.34      1499
           9       0.67      0.41      0.51      1499
          10       0.17      0.13      0.15      1499
          11       0.09      0.08      0.08      1499
          12       0.06      0.02      0.03      1499
          13       0.12      0.08      0.10      1499
          14       0.15      0.04      0.07      1499
          15       0.18      0.19      0.18      1499
          16       0.16      0.14      0.15      1499
          17       0.42    

## 0-16

In [17]:
xtrain16=xtrain.iloc[:,:16]
xtest16=xtest.iloc[:,:16]
xvalid16=xvalid.iloc[:,:16]

In [18]:
xtrain16['id']=ytrain
xtest16['id']=ytest
xvalid16['id']=yvalid
display(xtrain)

/tmp/ipykernel_78467/4039718790.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtrain16['id']=ytrain
/tmp/ipykernel_78467/4039718790.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtest16['id']=ytest
/tmp/ipykernel_78467/4039718790.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returni

,Channel_0,Channel_1,Channel_2,Channel_3,Channel_4,Channel_5,Channel_6,Channel_7,Channel_8,Channel_9,...,Channel_54,Channel_55,Channel_56,Channel_57,Channel_58,Channel_59,Channel_60,Channel_61,Channel_62,Channel_63
0,-0.000005,0.000002,0.000037,0.000039,0.000030,0.000026,-0.000016,-0.000014,0.000004,0.000018,...,-0.000021,-0.000008,-0.000035,-0.000045,-0.000066,-0.000039,-0.000033,-0.000048,-0.000039,-0.000039
1,-0.000012,-0.000024,0.000001,-0.000002,-0.000015,-0.000022,-0.000055,-0.000036,-0.000027,-0.000025,...,-0.000050,-0.000040,-0.000068,-0.000065,-0.000084,-0.000052,-0.000021,-0.000042,-0.000031,-0.000034
2,-0.000077,-0.000078,-0.000059,-0.000065,-0.000063,-0.000055,-0.000067,-0.000088,-0.000071,-0.000065,...,-0.000017,-0.000022,-0.000050,-0.000035,-0.000048,-0.000018,-0.000020,-0.000042,-0.000029,-0.000027
3,-0.000066,-0.000067,-0.000050,-0.000065,-0.000060,-0.000055,-0.000068,-0.000062,-0.000053,-0.000054,...,-0.000039,-0.000060,-0.000078,-0.000064,-0.000068,-0.000041,-0.000044,-0.000062,-0.000034,-0.000043
4,-0.000045,-0.000055,-0.000033,-0.000053,-0.000054,-0.000063,-0.000083,-0.000052,-0.000050,-0.000053,...,-0.000044,-0.000055,-0.000070,-0.000054,-0.000063,-0.000037,-0.000060,-0.000070,-0.000034,-0.000045
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899995,0.000055,0.000013,-0.000036,-0.000092,-0.000044,-0.000073,0.000019,0.000028,0.000032,-0.000043,...,0.000005,0.000019,-0.000023,-0.000062,-0.000021,-0.000005,0.000022,0.000022,-0.000018,0.000014
899996,-0.000050,-0.000065,-0.000089,-0.000119,-0.000064,-0.000116,-0.000026,-0.000044,-0.000070,-0.000080,...,-0.000007,-0.000014,-0.000038,-0.000075,-0.000048,-0.000031,-0.000008,0.000012,-0.000035,-0.000030
899997,0.000001,-0.000021,-0.000052,-0.000108,-0.000053,-0.000102,-0.000020,-0.000034,-0.000003,-0.000039,...,-0.000044,0.000030,0.000022,-0.000021,-0.000030,-0.000039,0.000026,0.000058,0.000017,0.000050
899998,0.000011,0.000051,0.000003,-0.000079,-0.000023,-0.000072,0.000031,-0.000010,0.000079,0.000008,...,-0.000064,0.000006,0.000007,-0.000009,-0.000026,-0.000063,0.000028,0.000044,-0.000009,0.000039


In [19]:
x16,y16=scale_dataset(xtrain16)
xt16,yt16=scale_dataset(xtest16)
xv16,yv16=scale_dataset(xvalid16)

In [20]:
model2=XGBClassifier(n_estimators=500)
model2.fit(x16,y16,early_stopping_rounds=10, eval_set=[(xv16, yv16)])
y_pred=model2.predict(xt16)
print(classification_report(yt16,y_pred))

/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-mlogloss:3.95584
[1]	validation_0-mlogloss:3.81972
[2]	validation_0-mlogloss:3.71704
[3]	validation_0-mlogloss:3.63810
[4]	validation_0-mlogloss:3.57185
[5]	validation_0-mlogloss:3.51312
[6]	validation_0-mlogloss:3.46801
[7]	validation_0-mlogloss:3.42156
[8]	validation_0-mlogloss:3.38207
[9]	validation_0-mlogloss:3.34842
[10]	validation_0-mlogloss:3.31633
[11]	validation_0-mlogloss:3.28849
[12]	validation_0-mlogloss:3.26423
[13]	validation_0-mlogloss:3.24068
[14]	validation_0-mlogloss:3.22118
[15]	validation_0-mlogloss:3.20052
[16]	validation_0-mlogloss:3.18306
[17]	validation_0-mlogloss:3.16494
[18]	validation_0-mlogloss:3.14867
[19]	validation_0-mlogloss:3.13299
[20]	validation_0-mlogloss:3.11670
[21]	validation_0-mlogloss:3.10015
[22]	validation_0-mlogloss:3.08734
[23]	validation_0-mlogloss:3.07533
[24]	validation_0-mlogloss:3.06128
[25]	validation_0-mlogloss:3.04646
[26]	validation_0-mlogloss:3.03483
[27]	validation_0-mlogloss:3.02275
[28]	validation_0-mlogloss:3.0

In [22]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model1 = Sequential(
    [
        Dense(16, activation = 'relu',   name = "L1"),
        Dense(256, activation = 'relu', name = "L2"),
        Dense(128, activation = 'relu', name = "L3"),
        Dense(64, activation = 'relu',   name = "L4"),
        Dense(32, activation = 'relu', name = "L5"),
        Dense(75, activation = 'linear', name = "L6"),
    ]
)

In [23]:
model1.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model1.fit(
    x16,y16,epochs=10,validation_data=(xv16,yv16)
)

Epoch 1/10


2023-10-04 13:47:22.263777: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 57600000 exceeds 10% of free system memory.


 7773/28125 [=======>......................] - ETA: 41s - loss: 2.7295

28125/28125 [==============================] - 61s 2ms/step - loss: 2.0060 - val_loss: 2.6217
Epoch 2/10
28125/28125 [==============================] - 62s 2ms/step - loss: 1.3554 - val_loss: 2.7408
Epoch 3/10
28125/28125 [==============================] - 62s 2ms/step - loss: 1.2092 - val_loss: 2.8221
Epoch 4/10
28125/28125 [==============================] - 62s 2ms/step - loss: 1.1365 - val_loss: 2.8277
Epoch 5/10
28125/28125 [==============================] - 62s 2ms/step - loss: 1.0877 - val_loss: 2.8175
Epoch 6/10
28125/28125 [==============================] - 62s 2ms/step - loss: 1.0532 - val_loss: 2.9038
Epoch 7/10
28125/28125 [==============================] - 61s 2ms/step - loss: 1.0269 - val_loss: 2.9518
Epoch 8/10
28125/28125 [==============================] - 63s 2ms/step - loss: 1.0060 - val_loss: 2.7804
Epoch 9/10
28125/28125 [==============================] - 61s 2ms/step - loss: 0.9892 - val_loss: 2.9476
Epoch 10/10
28125/28125 [==============================] - 64s 2ms

In [24]:
y_pred_classes = np.argmax(tf.nn.softmax(model1.predict(xt16)).numpy(),axis=1)
print(classification_report(yt16,y_pred_classes))

   1/3514 [..............................] - ETA: 4:20

3514/3514 [==============================] - 5s 1ms/step


2023-10-04 13:57:49.589696: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 33727500 exceeds 10% of free system memory.


              precision    recall  f1-score   support

           0       0.19      0.32      0.24      1499
           1       0.42      0.29      0.34      1499
           2       0.29      0.42      0.34      1499
           3       0.57      0.31      0.40      1499
           4       0.88      0.76      0.82      1499
           5       0.44      0.39      0.41      1499
           6       0.49      0.45      0.47      1499
           7       0.59      0.28      0.38      1499
           8       0.30      0.42      0.35      1499
           9       0.53      0.66      0.58      1499
          10       0.18      0.13      0.15      1499
          11       0.39      0.40      0.39      1499
          12       0.29      0.27      0.28      1499
          13       0.20      0.21      0.20      1499
          14       0.18      0.15      0.17      1499
          15       0.51      0.25      0.33      1499
          16       0.21      0.17      0.19      1499
          17       0.51    

## 0-32

In [21]:
xtrain32=xtrain.iloc[:,:32]
xtest32=xtest.iloc[:,:32]
xvalid32=xvalid.iloc[:,:32]

In [22]:
xtrain32['id']=ytrain
xtest32['id']=ytest
xvalid32['id']=yvalid

/tmp/ipykernel_78467/1675347936.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtrain32['id']=ytrain
/tmp/ipykernel_78467/1675347936.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtest32['id']=ytest
/tmp/ipykernel_78467/1675347936.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returni

In [23]:
x32,y32=scale_dataset(xtrain32)
xt32,yt32=scale_dataset(xtest32)
xv32,yv32=scale_dataset(xvalid32)

In [24]:
model3=XGBClassifier(n_estimators=500)
model3.fit(x32,y32,early_stopping_rounds=10, eval_set=[(xv32, yv32)])
y_pred=model3.predict(xt32)
print(classification_report(yt32,y_pred))

/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-mlogloss:3.79791
[1]	validation_0-mlogloss:3.57443
[2]	validation_0-mlogloss:3.41813
[3]	validation_0-mlogloss:3.29974
[4]	validation_0-mlogloss:3.20704
[5]	validation_0-mlogloss:3.12124
[6]	validation_0-mlogloss:3.05021
[7]	validation_0-mlogloss:2.98905
[8]	validation_0-mlogloss:2.93709
[9]	validation_0-mlogloss:2.88638
[10]	validation_0-mlogloss:2.84592
[11]	validation_0-mlogloss:2.80431
[12]	validation_0-mlogloss:2.76730
[13]	validation_0-mlogloss:2.73201
[14]	validation_0-mlogloss:2.69951
[15]	validation_0-mlogloss:2.66902
[16]	validation_0-mlogloss:2.63944
[17]	validation_0-mlogloss:2.61035
[18]	validation_0-mlogloss:2.58382
[19]	validation_0-mlogloss:2.55811
[20]	validation_0-mlogloss:2.53627
[21]	validation_0-mlogloss:2.51469
[22]	validation_0-mlogloss:2.49290
[23]	validation_0-mlogloss:2.47358
[24]	validation_0-mlogloss:2.45639
[25]	validation_0-mlogloss:2.43897
[26]	validation_0-mlogloss:2.41846
[27]	validation_0-mlogloss:2.39989
[28]	validation_0-mlogloss:2.3

In [28]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model2 = Sequential(
    [
        Dense(32, activation = 'relu',   name = "L1"),
        Dense(256, activation = 'relu', name = "L2"),
        Dense(128, activation = 'relu', name = "L3"),
        Dense(64, activation = 'relu',   name = "L4"),
        Dense(32, activation = 'relu', name = "L5"),
        Dense(75, activation = 'linear', name = "L6"),
    ]
)

In [29]:
model2.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model2.fit(
    x32,y32,epochs=10,validation_data=(xv32,yv32)
)

Epoch 1/10
28125/28125 [==============================] - 67s 2ms/step - loss: 1.4385 - val_loss: 2.0294
Epoch 2/10
28125/28125 [==============================] - 54s 2ms/step - loss: 0.6407 - val_loss: 2.0113
Epoch 3/10
28125/28125 [==============================] - 47s 2ms/step - loss: 0.4818 - val_loss: 1.8551
Epoch 4/10
28125/28125 [==============================] - 48s 2ms/step - loss: 0.4102 - val_loss: 1.9687
Epoch 5/10
28125/28125 [==============================] - 47s 2ms/step - loss: 0.3693 - val_loss: 1.8745
Epoch 6/10
28125/28125 [==============================] - 49s 2ms/step - loss: 0.3384 - val_loss: 1.9664
Epoch 7/10
28125/28125 [==============================] - 49s 2ms/step - loss: 0.3157 - val_loss: 2.0236
Epoch 8/10
28125/28125 [==============================] - 49s 2ms/step - loss: 0.2972 - val_loss: 1.9192
Epoch 9/10
28125/28125 [==============================] - 50s 2ms/step - loss: 0.2825 - val_loss: 2.0627
Epoch 10/10
28125/28125 [==============================

In [30]:
y_pred_classes = np.argmax(tf.nn.softmax(model2.predict(xt32)).numpy(),axis=1)
print(classification_report(yt32,y_pred_classes))

3514/3514 [==============================] - 3s 884us/step
              precision    recall  f1-score   support

           0       0.64      0.70      0.67      1499
           1       0.77      0.83      0.80      1499
           2       0.54      0.79      0.64      1499
           3       0.69      0.27      0.39      1499
           4       0.96      0.94      0.95      1499
           5       0.98      0.68      0.80      1499
           6       0.94      0.70      0.80      1499
           7       0.86      0.74      0.80      1499
           8       0.55      0.74      0.63      1499
           9       0.67      0.79      0.72      1499
          10       0.71      0.60      0.65      1499
          11       0.98      0.91      0.94      1499
          12       0.66      0.80      0.72      1499
          13       0.71      0.69      0.70      1499
          14       0.58      0.59      0.59      1499
          15       0.78      0.31      0.45      1499
          16       0.5

## 0-64

In [25]:
xtrain['id']=ytrain
xtest['id']=ytest
xvalid['id']=yvalid

In [26]:
x,y=scale_dataset(xtrain)
xt,yt=scale_dataset(xtest)
xv,yv=scale_dataset(xvalid)

In [27]:
model4=XGBClassifier(n_estimators=500)
model4.fit(x,y,early_stopping_rounds=10, eval_set=[(xv, yv)])
y_pred=model4.predict(xt)
print(classification_report(yt,y_pred))

/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-mlogloss:3.52774
[1]	validation_0-mlogloss:3.26387
[2]	validation_0-mlogloss:3.09289
[3]	validation_0-mlogloss:2.95280
[4]	validation_0-mlogloss:2.83453
[5]	validation_0-mlogloss:2.73392
[6]	validation_0-mlogloss:2.65230
[7]	validation_0-mlogloss:2.57743
[8]	validation_0-mlogloss:2.51035
[9]	validation_0-mlogloss:2.45098
[10]	validation_0-mlogloss:2.39400
[11]	validation_0-mlogloss:2.34369
[12]	validation_0-mlogloss:2.29405
[13]	validation_0-mlogloss:2.25297
[14]	validation_0-mlogloss:2.20795
[15]	validation_0-mlogloss:2.17071
[16]	validation_0-mlogloss:2.13244
[17]	validation_0-mlogloss:2.10075
[18]	validation_0-mlogloss:2.06592
[19]	validation_0-mlogloss:2.03720
[20]	validation_0-mlogloss:2.00864
[21]	validation_0-mlogloss:1.98550
[22]	validation_0-mlogloss:1.96018
[23]	validation_0-mlogloss:1.93251
[24]	validation_0-mlogloss:1.91045
[25]	validation_0-mlogloss:1.89048
[26]	validation_0-mlogloss:1.86718
[27]	validation_0-mlogloss:1.84779
[28]	validation_0-mlogloss:1.8

In [33]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model3 = Sequential(
    [
        Dense(64, activation = 'relu',   name = "L1"),
        Dense(256, activation = 'relu', name = "L2"),
        Dense(128, activation = 'relu', name = "L3"),
        Dense(64, activation = 'relu',   name = "L4"),
        Dense(32, activation = 'relu', name = "L5"),
        Dense(75, activation = 'linear', name = "L6"),
    ]
)

In [34]:
model3.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model3.fit(
    x,y,epochs=10,validation_data=(xv,yv)
)

Epoch 1/10
 6915/28125 [======>.......................] - ETA: 31s - loss: 1.8721

28125/28125 [==============================] - 50s 2ms/step - loss: 1.0110 - val_loss: 1.8567
Epoch 2/10
28125/28125 [==============================] - 50s 2ms/step - loss: 0.3903 - val_loss: 1.7396
Epoch 3/10
28125/28125 [==============================] - 50s 2ms/step - loss: 0.2807 - val_loss: 1.6829
Epoch 4/10
28125/28125 [==============================] - 52s 2ms/step - loss: 0.2257 - val_loss: 1.7655
Epoch 5/10
28125/28125 [==============================] - 50s 2ms/step - loss: 0.1914 - val_loss: 1.4496
Epoch 6/10
28125/28125 [==============================] - 49s 2ms/step - loss: 0.1671 - val_loss: 1.4852
Epoch 7/10
28125/28125 [==============================] - 51s 2ms/step - loss: 0.1486 - val_loss: 1.6931
Epoch 8/10
28125/28125 [==============================] - 50s 2ms/step - loss: 0.1344 - val_loss: 1.4802
Epoch 9/10
28125/28125 [==============================] - 51s 2ms/step - loss: 0.1231 - val_loss: 1.4067
Epoch 10/10
28125/28125 [==============================] - 51s 2ms

In [35]:
y_pred_classes = np.argmax(tf.nn.softmax(model3.predict(xt)).numpy(),axis=1)
print(classification_report(yt,y_pred_classes))

  91/3514 [..............................] - ETA: 3s

3514/3514 [==============================] - 4s 1ms/step
              precision    recall  f1-score   support

           0       0.86      0.83      0.85      1499
           1       0.76      0.93      0.83      1499
           2       0.70      0.69      0.70      1499
           3       0.93      0.80      0.86      1499
           4       0.96      0.99      0.97      1499
           5       0.99      0.84      0.91      1499
           6       0.91      0.93      0.92      1499
           7       0.96      0.77      0.85      1499
           8       0.57      0.87      0.69      1499
           9       0.72      0.91      0.80      1499
          10       0.70      0.62      0.66      1499
          11       0.98      0.83      0.90      1499
          12       0.89      0.73      0.80      1499
          13       0.53      0.78      0.63      1499
          14       0.84      0.74      0.79      1499
          15       0.88      0.75      0.81      1499
          16       0.85 